# Churn (attrition) de clients Telecom

On veut prédire le départ de clients d'un opérateur telecom à partir de données comme la formule d'abonnement, ou le temps de communication consommé.

On peut trouver le dataset sur :  
https://www.kaggle.com/becksddf/churn-in-telecoms-dataset  


## Librairies et fonctions utiles

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from IPython.core.display import HTML # permet d'afficher du code html dans jupyter

Fonction pour standardiser les données quantitatives (cont_feat est une liste des colonnes correspondant à des caractéristiques quantitatives) :

In [ ]:
def scale_feat(df,cont_feat) :
    df1=df
    scaler = preprocessing.RobustScaler()
    df1[cont_feat] = scaler.fit_transform(df1[cont_feat])
    scaler = preprocessing.StandardScaler()
    df1[cont_feat] = scaler.fit_transform(df1[cont_feat]) 
    return df1

Fonction pour tracer les courbes d'apprentissage sur l'ensemble d'apprentissage et l'ensemble de validation :

In [ ]:
from sklearn.model_selection import learning_curve
def plot_learning_curve(est, X_train, y_train) :
    train_sizes, train_scores, test_scores = learning_curve(estimator=rf, X=X_train, y=y_train, train_sizes=np.linspace(0.1, 1.0, 10),
                                                        cv=5,
                                                        n_jobs=-1)
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    plt.figure(figsize=(8,10))
    plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='training accuracy')
    plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
    plt.plot(train_sizes, test_mean,color='green', linestyle='--',marker='s', markersize=5,label='validation accuracy')
    plt.fill_between(train_sizes,test_mean + test_std,test_mean - test_std,alpha=0.15, color='green')
    plt.grid(b='on')
    plt.xlabel('Number of training samples')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.ylim([0.6, 1.0])
    plt.show()

Fonction pour tracer la courbe ROC :

In [ ]:
def plot_roc_curve(est,X_test,y_test) :
    probas = est.predict_proba(X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
    roc_auc = auc(false_positive_rate, true_positive_rate)
    plt.figure(figsize=(8,8))
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'r--')        # plus mauvaise courbe
    plt.plot([0,0,1],[0,1,1],'g:')     # meilleure courbe
    plt.xlim([-0.05,1.2])
    plt.ylim([-0.05,1.2])
    plt.ylabel('Taux de vrais positifs')
    plt.xlabel('Taux de faux positifs')
    plt.show

Fonction pour équilibrer un dataframe *df* sur la colonne cible *target_col* avec la classe minoritaire *minority_class* :

In [ ]:
def undersample(df, target_col, minority_class) :
    df_minority = df[df[target_col] == minority_class]
    df_majority = df.drop(df_minority.index)
    ratio=len(df_minority)/len(df_majority)
    df_majority = df_majority.sample(frac=ratio)
    df1 = pd.concat((df_majority,df_minority), axis=0)
    return df1.sample(frac=1)

## Traitement du dataset

In [ ]:
df = pd.read_csv("../input/telecom-data/telecom_churn.csv")

In [ ]:
df.head()

In [ ]:
df.count()

In [ ]:
df.info()

In [ ]:
df.describe()

On supprime le numéro de téléphone qui n'apporte pas d'infos pertinentes a priori :

In [ ]:
df = df.drop(['phone number'], axis=1)

On mappe les valeurs de la colonne cible en 0/1 :

In [ ]:
df['churn'] = df['churn'].map({ False: 0, True: 1 })

In [ ]:
df.columns

In [ ]:
discr_feat = ['state', 'international plan', 'voice mail plan']
cont_feat = list(set(df.columns) - set(discr_feat)-{'churn'})

On convertit les catégories en étiquettes numériques :

In [ ]:
for col in discr_feat :
    df[col]=df[col].astype('category')
    df[col] = df[col].cat.codes
    df[col]=df[col].astype('int8')

In [ ]:
df.head()

On vérifie s'il y a des valeurs indéterminées dans le dataset :

In [ ]:
df.isnull().values.sum()

Les valeurs numériques ont des caractéristiques très différentes :

In [ ]:
df[cont_feat].describe()

On normalise ces valeurs :

In [ ]:
df=scale_feat(df,cont_feat)

In [ ]:
df[cont_feat].describe()

## Suréchantillonnage

Il y a beaucoup moins de clients qui partent que de clients qui restent (heureusement ...) :

In [ ]:
df.churn.value_counts()

On va rééquilibrer le dataset en sur-échantillonant la classe minoritaire :

La méthode SMOTE (Synthetic Minority Oversampling TEchnique) consiste à synthétiser des éléments pour la classe minoritaire, à partir de ceux qui existent déjà. Elle fonctionne en choisissant au hasard un point de la classe minoritaire et en calculant les k-voisins les plus proches pour ce point. Les points synthétiques sont ajoutés entre le point choisi et ses voisins.

<img src="https://raw.githubusercontent.com/rafjaa/machine_learning_fecib/master/src/static/img/smote.png">

In [ ]:
df_test = undersample(df, 'churn', 1).sample(frac=0.2)

In [ ]:
X_test = df_test.drop(['churn'], axis=1)
y_test = df_test.churn
X_test = X_test.values
y_test = y_test.values

In [ ]:
df_train = df.drop(df_test.index)

In [ ]:
X = df_train.drop(['churn'], axis=1)
y = df_train.churn

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(ratio='minority')
X_train, y_train = smote.fit_sample(X, y)

## Machine learning

### Forêts aléatoires

On teste les forêts aléatoires :

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

In [ ]:
plot_learning_curve(rf, X_train, y_train)

In [ ]:
plot_roc_curve(rf,X_test,y_test)

### Gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
gb.fit(X_train,y_train)
y_gb = gb.predict(X_test)
cm = metrics.confusion_matrix(y_test, y_gb)
print(cm)
gb_score = metrics.accuracy_score(y_test, y_gb)
print(gb_score)
print(classification_report(y_test, y_gb))

In [ ]:
plot_learning_curve(gb, X_train, y_train)
plot_roc_curve(gb,X_test,y_test)

### XGBoost

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
print(xgb.score(X_test,y_test))

In [ ]:
y_xgb = xgb.predict(X_test)
cm = metrics.confusion_matrix(y_test, y_xgb)
print(cm)
xgb_score = metrics.accuracy_score(y_test, y_xgb)
print(xgb_score)

In [ ]:
plot_learning_curve(xgb, X_train, y_train)
plot_roc_curve(xgb,X_test,y_test)

In [ ]:
print(classification_report(y_test, y_xgb))